# Механизми пажње и трансформери

Један од главних недостатака рекурентних мрежа је то што све речи у низу имају исти утицај на резултат. Ово доводи до субоптималних перформанси код стандардних LSTM модела енкодера-декодера за задатке претварања секвенци у секвенце, као што су препознавање именованих ентитета и машински превод. У стварности, одређене речи у улазној секвенци често имају већи утицај на излазну секвенцу од других.

Размотримо модел претварања секвенци у секвенце, као што је машински превод. Он се имплементира помоћу две рекурентне мреже, где једна мрежа (**енкодер**) компримује улазну секвенцу у скривено стање, а друга, **декодер**, развија то скривено стање у преведени резултат. Проблем са овим приступом је што завршно стање мреже тешко памти почетак реченице, што доводи до лошијег квалитета модела код дугих реченица.

**Механизми пажње** пружају начин да се тежински одреди контекстуални утицај сваког улазног вектора на сваку излазну предикцију РНН-а. Ово се имплементира стварањем пречица између међустојања улазног РНН-а и излазног РНН-а. На овај начин, приликом генерисања излазног симбола $y_t$, узимамо у обзир сва улазна скривена стања $h_i$, са различитим тежинским коефицијентима $\alpha_{t,i}$. 

![Слика која приказује модел енкодера/декодера са адитивним слојем пажње](../../../../../translated_images/sr/encoder-decoder-attention.7a726296894fb567.webp)
*Модел енкодера-декодера са механизмом адитивне пажње у [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитиран из [овог блога](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матрица пажње $\{\alpha_{i,j}\}$ представља степен у којем одређене улазне речи утичу на генерисање одређене речи у излазној секвенци. Испод је пример такве матрице:

![Слика која приказује пример поравнања које је пронашао RNNsearch-50, преузето из Bahdanau - arviz.org](../../../../../translated_images/sr/bahdanau-fig3.09ba2d37f202a6af.webp)

*Слика преузета из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Сл.3)*

Механизми пажње су одговорни за велики део тренутног или скоро тренутног стања уметности у обради природног језика. Међутим, додавање пажње значајно повећава број параметара модела, што је довело до проблема са скалирањем РНН-ова. Кључно ограничење скалирања РНН-ова је то што рекурентна природа модела отежава груписање и паралелизацију тренинга. У РНН-у сваки елемент секвенце мора бити обрађен редоследно, што значи да се не може лако паралелизовати.

Усвајање механизама пажње у комбинацији са овим ограничењем довело је до стварања садашњих трансформер модела, који представљају стање уметности, а које данас познајемо и користимо, од BERT-а до OpenGPT3.

## Трансформер модели

Уместо да преносе контекст сваке претходне предикције у следећи корак евалуације, **трансформер модели** користе **позиционе кодирања** и **пажњу** како би ухватили контекст датог улаза у оквиру задатог прозора текста. Слика испод приказује како позициона кодирања са пажњом могу ухватити контекст унутар датог прозора.

![Анимирани GIF који приказује како се евалуације изводе у трансформер моделима.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Пошто се свака улазна позиција независно мапира на сваку излазну позицију, трансформери могу боље паралелизовати обраду од РНН-ова, што омогућава много веће и изражајније језичке моделе. Свака глава пажње може се користити за учење различитих односа између речи, што побољшава задатке обраде природног језика.

## Изградња једноставног трансформер модела

Keras не садржи уграђени слој трансформера, али можемо изградити сопствени. Као и раније, фокусираћемо се на класификацију текста из AG News скупа података, али вреди напоменути да трансформер модели показују најбоље резултате код сложенијих задатака обраде природног језика.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Нови слојеви у Keras-у треба да наследе класу `Layer` и имплементирају метод `call`. Хајде да почнемо са слојем **Positional Embedding**. Користићемо [неколико линија кода из званичне Keras документације](https://keras.io/examples/nlp/text_classification_with_transformer/). Претпоставићемо да свим улазним секвенцама додајемо попуне до дужине `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Овај слој се састоји од два `Embedding` слоја: за уграђивање токена (на начин који смо раније објаснили) и позиција токена. Позиције токена се креирају као низ природних бројева од 0 до `maxlen` користећи `tf.range`, а затим се прослеђују кроз слој за уграђивање. Два добијена вектора уграђивања се затим сабирају, производећи позиционо-уграђену репрезентацију улаза облика `maxlen`$\times$`embed_dim`.

Сада, хајде да имплементирамо трансформер блок. Он ће примати излаз претходно дефинисаног слоја за уграђивање:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Сада смо спремни да дефинишемо комплетан трансформер модел:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Модели

**BERT** (Bidirectional Encoder Representations from Transformers) је веома велики трансформер мрежни модел са више слојева, који има 12 слојева за *BERT-base* и 24 за *BERT-large*. Модел се прво претходно тренира на великом корпусу текстуалних података (Википедија + књиге) користећи несупервизирано учење (предвиђање маскираних речи у реченици). Током претходног тренирања, модел усваја значајан ниво разумевања језика, који се затим може искористити са другим скуповима података кроз фино подешавање. Овај процес се назива **трансферно учење**.

![слика са http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sr/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Постоји много варијација трансформер архитектура, укључујући BERT, DistilBERT, BigBird, OpenGPT3 и друге, које се могу фино подесити.

Хајде да видимо како можемо користити претходно тренирани BERT модел за решавање нашег традиционалног проблема класификације секвенци. Позваћемо се на идеје и део кода из [званичне документације](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

За учитавање претходно тренираних модела, користићемо **Tensorflow hub**. Прво, хајде да учитамо векторизатор специфичан за BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Важно је да користите исти векторизатор који је коришћен за обуку оригиналне мреже. Такође, BERT векторизатор враћа три компоненте:
* `input_word_ids`, што је секвенца бројева токена за улазну реченицу
* `input_mask`, која показује који део секвенце садржи стварни улаз, а који је попуњавање. Слична је маски коју производи слој `Masking`
* `input_type_ids` се користи за задатке моделирања језика и омогућава спецификацију две улазне реченице у једној секвенци.

Затим можемо инстанцирати BERT екстрактор карактеристика:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Дакле, BERT слој враћа неколико корисних резултата:
* `pooled_output` је резултат просека свих токена у секвенци. Можете га посматрати као интелигентно семантичко уграђивање целе мреже. Еквивалентан је излазу слоја `GlobalAveragePooling1D` у нашем претходном моделу.
* `sequence_output` је излаз последњег трансформер слоја (одговара излазу `TransformerBlock` у нашем горњем моделу).
* `encoder_outputs` су излази свих трансформер слојева. Пошто смо учитали BERT модел са 4 слоја (као што вероватно можете претпоставити из назива, који садржи `4_H`), он има 4 тензора. Последњи је исти као `sequence_output`.

Сада ћемо дефинисати модел за класификацију од краја до краја. Користићемо *функционалну дефиницију модела*, где дефинишемо улаз модела, а затим пружамо низ израза за израчунавање његовог излаза. Такође ћемо учинити тежине BERT модела не-тренирајућим и тренирати само завршни класификатор:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Иако има мало параметара који се могу тренирати, процес је прилично спор, јер је BERT екстрактор карактеристика рачунски захтеван. Изгледа да нисмо успели да постигнемо разумну тачност, било због недостатка тренинга или недостатка параметара модела.

Хајде да покушамо да "одмрзнемо" тежине BERT-а и тренирамо их такође. Ово захтева веома малу стопу учења, као и пажљивију стратегију тренинга са **загревањем** (warmup), користећи **AdamW** оптимизатор. Користићемо `tf-models-official` пакет за креирање оптимизатора:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Као што видите, обука иде прилично споро - али можда бисте желели да експериментишете и обучите модел неколико епоха (5-10) и видите да ли можете добити најбољи резултат у поређењу са приступима које смо раније користили.

## Huggingface Transformers библиотека

Још један веома чест (и мало једноставнији) начин коришћења Transformer модела је [HuggingFace пакет](https://github.com/huggingface/), који пружа једноставне грађевинске блокове за различите NLP задатке. Доступан је и за Tensorflow и за PyTorch, још један веома популаран оквир за неуронске мреже.

> **Напомена**: Ако нисте заинтересовани да видите како ради Transformers библиотека - можете прескочити до краја овог нотебука, јер нећете видети ништа суштински другачије од онога што смо већ урадили. Понављаћемо исте кораке обуке BERT модела користећи другу библиотеку и знатно већи модел. Стога, процес укључује прилично дугу обуку, па можда само желите да прегледате код.

Хајде да видимо како наш проблем може бити решен коришћењем [Huggingface Transformers](http://huggingface.co).


Прва ствар коју треба да урадимо је да изаберемо модел који ћемо користити. Поред неких уграђених модела, Huggingface садржи [онлајн репозиторијум модела](https://huggingface.co/models), где можете пронаћи много више унапред обучених модела које је креирала заједница. Сви ти модели могу се учитати и користити само пружањем имена модела. Сви потребни бинарни фајлови за модел биће аутоматски преузети.

У одређеним ситуацијама биће потребно да учитате сопствене моделе, у ком случају можете навести директоријум који садржи све релевантне фајлове, укључујући параметре за токенизатор, `config.json` фајл са параметрима модела, бинарне тежине, итд.

На основу имена модела, можемо инстанцирати и модел и токенизатор. Хајде да почнемо са токенизатором:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` објекат садржи `encode` функцију која се може директно користити за кодирање текста:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Можемо такође користити токенизер за кодирање секвенце на начин погодан за прослеђивање моделу, тј. укључујући `token_ids`, `input_mask` поља, итд. Можемо такође назначити да желимо Tensorflow тенсоре пружањем аргумента `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

У нашем случају, користићемо унапред обучени BERT модел под називом `bert-base-uncased`. *Uncased* означава да је модел неосетљив на велика и мала слова.

Приликом тренирања модела, потребно је да обезбедимо токенизовану секвенцу као улаз, и зато ћемо дизајнирати процесну линију за обраду података. Пошто је `tokenizer.encode` Python функција, користићемо исти приступ као у претходној јединици, позивајући је помоћу `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Сада можемо учитати стварни модел користећи пакет `BertForSequenceClassification`. Ово осигурава да наш модел већ има потребну архитектуру за класификацију, укључујући завршни класификатор. Видећете поруку упозорења која наводи да тежине завршног класификатора нису иницијализоване и да ће модел захтевати претходну обуку - то је потпуно у реду, јер је то управо оно што ћемо урадити!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Као што можете видети из `summary()`, модел садржи скоро 110 милиона параметара! Претпостављено, ако желимо једноставан задатак класификације на релативно малом скупу података, не желимо да тренирамо основни слој BERT-а:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Сада смо спремни да почнемо са обуком!

> **Напомена**: Обука БЕРТ модела у пуној скали може бити веома временски захтевна! Због тога ћемо га обучавати само за првих 32 серије. Ово је само да бисмо показали како је обука модела подешена. Ако сте заинтересовани за обуку у пуној скали - само уклоните параметре `steps_per_epoch` и `validation_steps`, и припремите се за чекање!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Ако повећате број итерација и сачекате довољно дуго, и обучавате током неколико епоха, можете очекивати да класификација помоћу BERT-а даје најбољу тачност! То је зато што BERT већ прилично добро разуме структуру језика, па је потребно само дорађивање завршног класификатора. Међутим, пошто је BERT велики модел, цео процес обучавања траје дуго и захтева озбиљну рачунарску снагу! (GPU, и пожељно више од једног).

> **Напомена:** У нашем примеру користимо један од најмањих унапред обучених BERT модела. Постоје већи модели који вероватно дају боље резултате.


## Закључак

У овој јединици смо видели веома недавне архитектуре модела засноване на **трансформерима**. Применили смо их за наш задатак класификације текста, али слично, BERT модели могу се користити за извлачење ентитета, одговарање на питања и друге NLP задатке.

Модели трансформера представљају тренутно најсавременије решење у NLP-у, и у већини случајева треба да буду прво решење са којим ћете почети експериментисање када имплементирате прилагођена NLP решења. Међутим, разумевање основних принципа рекурентних неуронских мрежа, о којима смо говорили у овом модулу, изузетно је важно ако желите да изградите напредне неуронске моделе.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
